# Analysing the data
## First, importing the data from SQL joined together with the relevant columns

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


In [2]:
load_dotenv()
password = os.getenv("DATABASE_PASSWORD")
db_name = os.getenv("DATABASE_NAME")
port = os.getenv("DATABASE_PORT")
hostname = os.getenv("DATABASE_HOSTNAME")
username = os.getenv("DATABASE_USERNAME")

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{db_name}")

In [8]:
# pollution: geo_code, pm25_2007, pm25_2024, pm10_2007, pm10__2024, no2_2007, no2_2024
# imd2007: join(msoa11_code), score
# imd2024: join(msoac), imd19_score, msoarank

query = "SELECT p.geo_code, p.geo_label, p.pm25_2007, p.pm25_2024, p.pm10_2007, p.pm10_2024, p.no2_2007, p.no2_2024, sub.imd07_score, sub.imd19_score, sub.msoarank as msoarank19 from pollution_msoa p left join (select i7.msoa11_code, i7.score as imd07_score, i9.imd19_score, i9.msoarank from imd2007 i7 left join imd2019 i9 on i7.msoa11_code = i9.msoac) sub on p.geo_code = sub.msoa11_code;"

df = pd.read_sql(query, con=engine)

df.head()

,geo_code,geo_label,pm25_2007,pm25_2024,pm10_2007,pm10_2024,no2_2007,no2_2024,imd07_score,imd19_score,msoarank19
0,E02002873,Nottingham 006,10.720420,7.702422,18.64338,11.95465,22.05027,13.084070,48.025117,37.601338,866.0
1,E02000793,Richmond upon Thames 010,13.415840,8.492700,23.05174,13.04231,31.02554,16.298560,11.179251,11.206442,5179.0
2,E02002661,Kingston upon Hull 010,9.300502,7.248457,16.61529,11.91518,17.33395,9.187561,39.290738,39.315680,760.0
3,E02000077,Bexley 013,12.638760,8.657308,22.06381,13.38199,28.51846,14.747010,13.831107,12.745682,4756.0
4,E02000818,Southwark 012,15.081210,9.819899,27.67150,16.87792,47.45792,24.163360,40.875765,34.455332,1086.0


In [9]:
# adding columns for the difference in pollution and imd.

df["delta_pm25"] = df["pm25_2024"] - df["pm25_2007"]
df["delta_pm10"] = df["pm10_2024"] - df["pm10_2007"]
df["delta_no2"] = df["no2_2024"] - df["no2_2007"]
df["delta_imd"] = df["imd19_score"] - df["imd07_score"]

In [13]:
df.loc[df.geo_label.str.contains("Blackpool")]

,geo_code,geo_label,pm25_2007,pm25_2024,pm10_2007,pm10_2024,no2_2007,no2_2024,imd07_score,imd19_score,msoarank19,delta_pm25,delta_pm10,delta_no2,delta_imd
11,E02002647,Blackpool 015,7.304089,5.563503,14.02882,8.399649,17.51231,6.714581,44.775313,52.031346,226.0,-1.740586,-5.629171,-10.797729,7.256033
57,E02002644,Blackpool 012,7.219594,5.478445,13.82172,8.257591,16.97446,6.652940,22.004915,27.395475,1888.0,-1.741149,-5.564129,-10.321520,5.390559
964,E02002645,Blackpool 013,6.859068,5.147429,12.80471,7.711064,13.20823,5.351843,70.551470,86.903304,1.0,-1.711639,-5.093646,-7.856387,16.351834
1154,E02002648,Blackpool 016,7.410446,5.543506,14.10895,8.251907,18.02631,6.270205,35.103357,42.392158,591.0,-1.866940,-5.857043,-11.756105,7.288802
1461,E02002636,Blackpool 004,7.452719,5.532287,14.15957,8.856679,18.51295,7.141951,28.525470,31.396162,1402.0,-1.920432,-5.302891,-11.370999,2.870692
1645,E02002633,Blackpool 001,6.735975,5.175716,12.62793,7.689661,12.90768,5.398451,20.015694,24.489817,2296.0,-1.560259,-4.938269,-7.509229,4.474122
1768,E02002649,Blackpool 017,6.776242,5.106966,12.57654,7.612202,12.91350,5.282820,33.492456,41.141985,660.0,-1.669276,-4.964338,-7.630680,7.649529
2096,E02002641,Blackpool 009,7.198890,5.420807,13.73135,8.065394,16.76780,6.350140,27.916356,33.133858,1212.0,-1.778083,-5.665956,-10.417660,5.217502
2239,E02002635,Blackpool 003,6.551899,4.966263,12.04563,7.378899,10.46675,4.415491,20.354552,26.279637,2048.0,-1.585636,-4.666731,-6.051259,5.925086
2955,E02002634,Blackpool 002,6.818038,5.285441,12.86089,7.790407,13.55405,5.940584,27.656402,31.813822,1369.0,-1.532597,-5.070483,-7.613466,4.157420
